In [1]:
import pandas as pd
import psycopg2
import numpy as np

In [2]:
books = pd.read_csv('api/data/books.csv')
users = pd.read_csv('api/data/users.csv')
ratings = pd.read_csv('api/data/ratings.csv')

In [3]:
books.replace({np.nan: None}, inplace=True), users.replace({np.nan: None}, inplace=True), ratings.replace({np.nan: None}, inplace=True)

(None, None, None)

In [4]:
print("Books Columns:", books.columns)
print("Users Columns:", users.columns)
print("Ratings Columns:", ratings.columns)

Books Columns: Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')
Users Columns: Index(['User-ID', 'Location', 'Age'], dtype='object')
Ratings Columns: Index(['User-ID', 'ISBN', 'Book-Rating'], dtype='object')


In [5]:
def normalize_column_names(df):
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('-', '_')
    return df

In [6]:
books = normalize_column_names(books)
users = normalize_column_names(users)
ratings = normalize_column_names(ratings)

In [7]:
books.rename(columns={'isbn': 'book_id'}, inplace=True)
ratings.rename(columns={'isbn': 'book_id'}, inplace=True)

In [8]:
print("Books Columns after normalization:", books.columns)
print("Users Columns after normalization:", users.columns)
print("Ratings Columns after normalization:", ratings.columns)

Books Columns after normalization: Index(['book_id', 'book_title', 'book_author', 'year_of_publication',
       'publisher', 'image_url_s', 'image_url_m', 'image_url_l'],
      dtype='object')
Users Columns after normalization: Index(['user_id', 'location', 'age'], dtype='object')
Ratings Columns after normalization: Index(['user_id', 'book_id', 'book_rating'], dtype='object')


**Books**
- primary_key: book_id

**Users**
- primary_key: user_id

**Rating**
- ref_key: user_id -> Users(user_id); book_id -> Books(book_id)

In [2]:
connection = psycopg2.connect(
    dbname='postgres',
    user='postgres',
    password='admin',
    host='localhost',
    port='5432'
)

In [3]:
def db_creation(connection: psycopg2.extensions.connection):

    books_query = """CREATE TABLE IF NOT EXISTS books (
        book_id VARCHAR PRIMARY KEY,
        title VARCHAR,
        author VARCHAR,
        year_of_publication INT,
        publisher VARCHAR,
        image_url_s VARCHAR,
        image_url_m VARCHAR,
        image_url_l VARCHAR
    );"""

    users_query = """CREATE TABLE IF NOT EXISTS users (
        user_id BIGINT PRIMARY KEY,
        password VARCHAR DEFAULT '12345678',
        location VARCHAR,
        age INT
    );"""

    ratings_query = """CREATE TABLE IF NOT EXISTS ratings (
        user_id BIGINT,
        book_id VARCHAR,
        rating INT,
        PRIMARY KEY (user_id, book_id),
        FOREIGN KEY (user_id) REFERENCES users(user_id),
        FOREIGN KEY (book_id) REFERENCES books(book_id)
    );"""

    with connection.cursor() as cursor:
        cursor.execute(books_query)
        cursor.execute(users_query)
        cursor.execute(ratings_query)
        connection.commit()

In [11]:
db_creation(connection)

In [12]:
def insert_books(connection: psycopg2.extensions.connection, df: pd.DataFrame):
    tuples = [tuple(x) for x in df.to_numpy()]
    table_columns = ['book_id', 'title', 'author', 'year_of_publication', 'publisher', 'image_url_s', 'image_url_m', 'image_url_l']
    cols = ','.join(table_columns)
    placeholders = ','.join(['%s'] * len(table_columns))
    query = f"INSERT INTO books ({cols}) VALUES ({placeholders}) ON CONFLICT DO NOTHING;"

    with connection.cursor() as cursor:
        cursor.executemany(query, tuples)
        connection.commit()

In [13]:
def insert_users(connection: psycopg2.extensions.connection, df: pd.DataFrame):
    tuples = [tuple(x) for x in df.to_numpy()]
    table_columns = ['user_id', 'location', 'age']
    cols = ','.join(table_columns)
    placeholders = ','.join(['%s'] * len(table_columns))
    query = f"INSERT INTO users ({cols}) VALUES ({placeholders}) ON CONFLICT DO NOTHING;"

    with connection.cursor() as cursor:
        cursor.executemany(query, tuples)
        connection.commit()

In [14]:
def insert_rating(connection: psycopg2.extensions.connection, df: pd.DataFrame):
    tuples = [tuple(x) for x in df.to_numpy()]
    table_columns = ['user_id', 'book_id', 'rating']
    cols = ','.join(table_columns)
    placeholders = ','.join(['%s'] * len(table_columns))
    query = f"INSERT INTO ratings ({cols}) VALUES ({placeholders}) ON CONFLICT DO NOTHING;"

    with connection.cursor() as cursor:
        cursor.executemany(query, tuples)
        connection.commit()


In [ ]:
insert_books(connection, books)

In [ ]:
insert_users(connection, users)

In [15]:
insert_rating(connection, ratings)